<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import PyPDF2

In [3]:
reader = PyPDF2.PdfReader('data/Thygesen et al 1993.pdf')

In [10]:
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

In [26]:
# Skip the introduction / table of contents
chunk1 = reader.pages[14:27]

# There are three papers in French; skip these.
chunk2 = reader.pages[37:111]
chunk3 = reader.pages[145:378]
chunk4 = reader.pages[391:434]

In [27]:
content = []
for chunk in [chunk1, chunk2, chunk3, chunk4]:
    for page in chunk:
        text = page.extract_text()
        content.append(text)

In [106]:
from unidecode import unidecode_expect_ascii
import re
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import everygrams, padded_everygram_pipeline

In [32]:
KEEP_PUNCT = '''.?!'"$%'''
PUNCT = ''.join([x for x in punctuation if x not in KEEP_PUNCT])

In [64]:
# Copied from the main script so preprocessing matches what I'm using
# in the langauge model.

def clean_text(s):
    lower = s.lower()
    decoded  = unidecode_expect_ascii(lower)
    no_punct = re.sub('[' + re.escape(PUNCT) + '\n\t' + ']', ' ', decoded)
    pad_numbers = re.sub('\d', ' \g<0> ', no_punct)
    reduce_spaces = re.sub(' +', ' ', pad_numbers)
    return reduce_spaces

In [95]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [65]:
clean_content = [clean_text(page) for page in content]

In [98]:
corpus = nlp(' '.join(clean_content))

In [104]:
sentences = [sent.text.strip() for sent in corpus.sents]

In [120]:
unigrams, _ = padded_everygram_pipeline(1, [word_tokenize(s) for s in sentences])
unigrams = [w for s in list(unigrams) for w in list(s)] 
bigrams, _ = padded_everygram_pipeline(2, [word_tokenize(s) for s in sentences])
bigrams = [w for s in list(bigrams) for w in list(s) if len(w) == 2]

In [121]:
from collections import Counter
word_counts = Counter(unigrams)

In [122]:
import pandas as pd
df_unigrams = pd.Series(word_counts)
df_unigrams.name = 'Unigrams'

In [125]:
df_unigrams.to_csv('data/unigram_counts.csv')

In [126]:
bigram_counts = Counter(bigrams)

In [127]:
df_bigrams = pd.Series(bigram_counts)
df_bigrams.name = 'Bigrams'

In [130]:
df_bigrams.to_csv('data/bigram_counts.csv')